In [1]:
import json
import logging
import sys
from datetime import datetime
from pathlib import Path
from tempfile import TemporaryDirectory
from types import SimpleNamespace

import arcgis
from palletjack import FeatureServiceInlineUpdater, GSheetLoader
from palletjack.transform import APIGeocoder
from supervisor.message_handlers import SendGridHandler
from supervisor.models import MessageDetails, Supervisor

try:
    from . import config, version
except ImportError:
    import config
    import version

cwd = Path.cwd()
print(cwd)
print(type(cwd))

secret_folder = cwd / "secrets"
secrets = json.loads((secret_folder / "secrets.json").read_text(encoding="utf-8"))

ModuleNotFoundError: No module named 'palletjack'

In [164]:
gis = arcgis.gis.GIS(config.AGOL_ORG, secrets["AGOL_USER"], secrets["AGOL_PASSWORD"])

feature_layer_item = gis.content.get(config.FEATURE_LAYER_ITEMID)
feature_layer = arcgis.features.FeatureLayer.fromitem(feature_layer_item)
featureset = feature_layer.query()
live_dataframe = featureset.sdf

In [129]:
live_dataframe.spatial.validate()

True

In [108]:
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'

print("Reading in DABS data from Google Sheet ...")
loader = GSheetLoader(secrets["SERVICE_ACCOUNT_JSON"])
dabs_df = loader.load_specific_worksheet_into_dataframe(secrets["SHEET_ID"], "TEST_TAB", by_title=True)

#: Geocode rows if ACTION.casefold() == 'add'
print("Geocoding new rows ...")
adds_df = dabs_df[dabs_df["ACTION"].str.casefold() == "add"]
adds_df.drop(["ACTION"], axis="columns", inplace=True)
removes_df = dabs_df[dabs_df["ACTION"].str.casefold() == "remove"]

geocoder = APIGeocoder(secrets["GEOCODE_KEY"])
geo_df = geocoder.geocode_dataframe(adds_df, "Address", "Zip", 4326, rate_limits=(0.015, 0.03), acceptScore=90)
valid = geo_df.spatial.validate()
print(f"Is geo_df spatial?: {valid}")

Reading in DABS data from Google Sheet ...
Geocoding new rows ...
Is geo_df spatial?: True


In [130]:
live_dataframe.head(2)

,OBJECTID,County,Suite_Unit,Lic_Type,Lic_Number,Lic_Descr,Name,Address,City,Zip,Renew_Date,Lic_Group,Comp_Zone,Comp_Group,Comp_Needed,Point_X,Point_Y,SHAPE
0,1,SALT LAKE,None,OP,OP00476,OFF PREMISE BEER RETAILER,WHOLE FOODS MARKET (SALT LAKE),1131 E WILMINGTON AVE,SALT LAKE CITY,84123,2/28,Off-Premise,4,F,no,-111.858739,40.723633,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
1,2,CACHE,None,OP,OP00594,OFF PREMISE BEER RETAILER,TOP STOP CONVENIENCE STORE (LOGAN),970 N MAIN,LOGAN,84321,2/28,Off-Premise,10,A,no,-111.834110,41.749773,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."


In [151]:
geo_df.head(2)

,Lic_Number,Name,Address,City,Zip,x,y,score,matchAddress,SHAPE
0,RL01498,CRUMB BROTHERS,291 S 300 W,LOGAN,84321,-111.843346,41.726437,100.0,"291 S 300 W, LOGAN","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
1,RL01499,SOCIAL DISTRICT BISTRO,250 RED CLIFFS DR,ST. GEORGE,84790,-113.553374,37.110635,100.0,"250 RED CLIFFS DR, ST GEORGE","{""spatialReference"": {""wkid"": 4326}, ""x"": -113..."


In [127]:
print(live_dataframe.iloc[0]["SHAPE"])
len(live_dataframe.index)

{'x': 427478.4543757879, 'y': 4508432.323816502, 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}


4296

In [128]:
pd.DataFrame.spatial.from_xy(df=live_dataframe, x_column="Point_X", y_column="Point_Y", sr=4326)
print(live_dataframe.iloc[0]["SHAPE"])
len(live_dataframe.index)

{'spatialReference': {'wkid': 4326}, 'x': -111.85873939, 'y': 40.7236328}


4296

In [59]:
print(geo_df.iloc[0]["SHAPE"])
len(geo_df.index)

{'spatialReference': {'wkid': 3857}, 'x': -12450344.320418216, 'y': 5120088.737326822}


17

In [96]:
working = live_dataframe.head(10)
working.head()

sref = arcgis.geometry.SpatialReference(3857)
# arcgis.geometry.project(geometries=working, in_sr=26912, out_sr=sref)

print(working.iloc[0]["SHAPE"])
print(working.iloc[0]["Point_X"])
print(working.iloc[0]["Point_Y"])

{'x': 427478.4543757879, 'y': 4508432.323816502, 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}
-111.85873939
40.7236328


In [83]:
sref = arcgis.geometry.SpatialReference(3857)
arcgis.geometry.project(geometries=working["SHAPE"], in_sr=26912, out_sr=sref)

# sedf = pd.DataFrame.spatial.from_xy(df=live_dataframe, x_column='Point_X', y_column='Point_Y', sr=4326)
# sedf.iloc[0]['SHAPE']

# m = gis.map()
# m

print(reprojected.iloc[0]["SHAPE"])

Exception: Invalid or missing input parameters.
The operation was attempted on an empty geometry.
null
(Error Code: 400)

In [113]:
live_dataframe.spatial.validate()

True

In [162]:
combined = pd.concat([live_dataframe, geo_df])
combined.drop(["x", "y", "score", "matchAddress"], axis=1, inplace=True)

combined["Comp_Zone"].fillna(0, inplace=True)
combined["Comp_Zone"] = combined["Comp_Zone"].astype("int")

combined.tail(50)

,OBJECTID,County,Suite_Unit,Lic_Type,Lic_Number,Lic_Descr,Name,Address,City,Zip,Renew_Date,Lic_Group,Comp_Zone,Comp_Group,Comp_Needed,Point_X,Point_Y,SHAPE
4263,4271,CACHE,None,RL,RL00371,RESTAURANT LIMITED,TAKARA SUSHI JAPANESE RESTAURANT,55 W 1000 N #1,LOGAN,84321,10/31,Restaurant,10,A,yes,-111.836177,41.750526,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4264,4272,EMERY,None,RL,RL00863,RESTAURANT LIMITED,TAMARISK RESTAURANT,1710 E MAIN ST,GREEN RIVER,84525,10/31,Restaurant,14,F,yes,-110.141991,38.993144,"{""spatialReference"": {""wkid"": 4326}, ""x"": -110..."
4265,4273,UTAH,None,RL,RL01339,RESTAURANT LIMITED,TAMASHI RAMEN & SUSHI BAR,466 N 900 W,AMERICAN FORK,84003,10/31,Restaurant,12,D,yes,-111.820271,40.387060,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4266,4274,SALT LAKE,None,RL,RL00444,RESTAURANT LIMITED,TANDOOR,733 E 3300 S,SALT LAKE CITY,84106,10/31,Restaurant,7,E,yes,-111.870108,40.700310,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4267,4275,UTAH,None,RL,RL01331,RESTAURANT LIMITED,TANDOOR INDIAN GRILL (PROVO),1600 N FREEDOM BLVD,PROVO,84604,10/31,Restaurant,15,E,yes,-111.661717,40.255809,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4268,4276,CACHE,None,RL,RL00537,RESTAURANT LIMITED,TANDOORI OVEN,720 E 1000 N,LOGAN,84321,10/31,Restaurant,10,A,yes,-111.816054,41.749754,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4269,4277,UTAH,None,RL,RL01324,RESTAURANT LIMITED,TAPA SUSHI JAPANESE CUISINE,513 S 500 E STE A,AMERICAN FORK,84003,10/31,Restaurant,12,D,yes,-111.784851,40.366678,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4270,4278,SALT LAKE,None,RL,RL01377,RESTAURANT LIMITED,TAPPO (RL),565 E 2100 S,SALT LAKE CITY,84106,10/31,Restaurant,4,F,yes,-111.874791,40.725641,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4271,4279,DAVIS,None,RL,RL00880,RESTAURANT LIMITED,TASTE OF INDIA,1664 WOODLAND PARK DR STE 401 401,LAYTON,84041,10/31,Restaurant,11,B,yes,-111.984083,41.084552,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4272,4280,DAVIS,None,RL,RL00910,RESTAURANT LIMITED,TASTE OF INDIA GRILL,282 S 500 W STE A,BOUNTIFUL,84010,10/31,Restaurant,3,B,yes,-111.892411,40.886494,"{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."


In [131]:
print(combined["OBJECTID"].max())

4303.0


In [155]:
# live_dataframe = live_dataframe.astype({"OBJECTID": int})
# geo_df['OBJECTID'] = 0
max_OID = live_dataframe["OBJECTID"].max()

print(max_OID)

geo_df.insert(0, "OBJECTID", range(max_OID + 1, max_OID + len(geo_df) + 1))

len(geo_df.index)
geo_df.head(20)

4303


,OBJECTID,Lic_Number,Name,Address,City,Zip,x,y,score,matchAddress,SHAPE
0,4304,RL01498,CRUMB BROTHERS,291 S 300 W,LOGAN,84321,-111.843346,41.726437,100.00,"291 S 300 W, LOGAN","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
1,4305,RL01499,SOCIAL DISTRICT BISTRO,250 RED CLIFFS DR,ST. GEORGE,84790,-113.553374,37.110635,100.00,"250 RED CLIFFS DR, ST GEORGE","{""spatialReference"": {""wkid"": 4326}, ""x"": -113..."
2,4306,RE03053,MR & MRS CRAB JUICY SEAFOOD,4150 S REDWOOD RD,SLC,84123,-111.939222,40.680960,100.00,"4150 S REDWOOD RD, SALT LAKE CITY","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
3,4307,BE02740,BLUES AT THE DEPOT,13 N 400 W,SLC,84101,-111.902971,40.769724,100.00,"13 N 400 W, SALT LAKE CITY","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
4,4308,RE03057,HEIRLOOM COMMON,195 W MAIN ST,MIDWAY,84049,-111.476412,40.512102,100.00,"195 W MAIN ST, Midway","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
5,4309,RE03054,MEAT HOOK BBQ,3380 S REDWOOD RD,SLC,84119,-111.939269,40.698975,100.00,"3380 S REDWOOD RD, SALT LAKE CITY","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
6,4310,HL00052,AC MARRIOTT SLC DOWNTOWN,225 W 200 S,SLC,84101,-111.897695,40.764778,100.00,"225 W 200 S, SALT LAKE CITY","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
7,4311,RE03055,TOPGOLF,484 N 700 E,VINEYARD,84057,-111.678873,40.306057,100.00,"484 N 700 E, Orem","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
8,4312,MR00225,SCOTT RICHEY,14301 N COUNCIL FIRE TRAIL,KAMAS,84036,-111.431921,40.681506,100.00,"14301 N COUNCIL FIRE TRL, HEBER CITY","{""spatialReference"": {""wkid"": 4326}, ""x"": -111..."
9,4313,MW00040,EVERYTHING GOOD NURTITION,1630 S 4650 W,SLC,84104,-112.003048,40.735193,100.00,"1630 S 4650 W, SALT LAKE CITY","{""spatialReference"": {""wkid"": 4326}, ""x"": -112..."


In [154]:
# geo_df.drop('OBJECTID', axis=1, inplace=True)

In [157]:
type(geo_df.iloc[1].OBJECTID)

numpy.int64

In [165]:
import time

month_tab = time.strftime("%m/%Y")
print(month_tab)

10/2022


In [167]:
str_date_added = time.strftime("%Y%m%d")
print(str_date_added)

20221005
